In [22]:
import pandas as pd
from IPython.display import display

# Function to download the model from the Hugging Face model hub
from huggingface_hub import hf_hub_download

# Importing the Llama class from the llama_cpp module
from llama_cpp import Llama

# Importing the json module
import json

In [21]:
# Installation for GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS" pip3 install llama-cpp-python --upgrade --force-reinstall --no-cache-dir

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 36.8 MB 10.7 MB/s eta 0:00:01     |███████████████████             | 22.0 MB 9.2 MB/s eta 0:00:02
  distutils: /private/var/folders/y9/8s_084891yq5xjgdqw12vk4m0000gn/T/pip-build-env-yne8tpio/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/y9/8s_084891yq5xjgdqw12vk4m0000gn/T/pip-build-env-yne8tpio/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  user = False
  home = None
  root = None
  prefix = '/private/var/folders/y9/8s_084891yq5xjgdqw12vk4m0000gn/T/pip-build-env-yne8tpio/normal'
  distutils: /private/var/folders/y9/8s_084891yq5xjgdqw12vk4m0000gn/T/pip-build-env-yne8tpio/overlay/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/y9/8s_084891yq5xjgdqw12vk4m0000gn/T/pip-build-env-yne8tpio/overlay/lib/python3.9/

In [16]:
# Install the hugging face hub
!pip3 install huggingface_hub -q

You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


# Support Ticket Categorization

## Data Overview

In [11]:
df = pd.read_csv("Support_ticket_text_data_mid_term.csv")
print(f"Shape of dataframe: {df.shape}")
display(df.head())
counter = 0
for i in df["support_ticket_text"].isna():
    if i:
        counter += 1

if counter:
    print("Missing values are present")
    print(f"Number of missing values is {counter}")
else:
    print("No missing values present")

Shape of dataframe: (21, 2)


,support_tick_id,support_ticket_text
0,ST2023-006,My internet connection has significantly slowe...
1,ST2023-007,Urgent help required! My laptop refuses to sta...
2,ST2023-008,I've accidentally deleted essential work docum...
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...
4,ST2023-010,"My smartphone battery is draining rapidly, eve..."


No missing values present


## Model Building

In [12]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_M.gguf" # the model is in gguf format

In [23]:
# Using hf_hub_download to download a model from the Hugging Face model hub
# The repo_id parameter specifies the model name or path in the Hugging Face repository
# The filename parameter specifies the name of the file to download
model_path = hf_hub_download(
    repo_id = model_name_or_path,    # complete the code to mentioned the repo_id
    filename = model_basename    # complete the code to mentioned the filename
)

llama-2-13b-chat.Q5_K_M.gguf:   0%|          | 0.00/9.23G [00:00<?, ?B/s]

In [25]:
# complete the code to create an instance of the 'Llama' class with specified parameters

lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2,  # CPU cores
    n_batch=512,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=43,  # Change this value based on your model and your GPU VRAM pool.
    n_ctx=4096,  # Context window
)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /Users/ChiragSP/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.

In [26]:
def generate_llama_response(instruction, review):

    # System message explicitly instructing not to include the review text
    system_message = """
        [INST]<<SYS>>
        {}
        <</SYS>>[/INST]
    """.format(instruction)

    # Combine user_prompt and system_message to create the prompt
    prompt = f"{review}\n{system_message}"

    # Generate a response from the LLaMA model
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=1024,
        temperature=0.01,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=['INST'],
        echo=False,
        seed=42,
    )

    # Extract the sentiment from the response
    response_text = response["choices"][0]["text"]
    return response_text

## Task 1: Ticket Categorization